In [1]:
import cv2
import os
import pandas as pd
import numpy as np

In [2]:
folder = 'NIHimages/'
image_name_list = os.listdir(folder)

In [3]:
demo_filename = 'Data_Entry_2017_v2020.csv'

In [4]:
n = len(image_name_list)
print(n)

4999


In [5]:
df = pd.read_csv(demo_filename)

In [6]:
df['Image Index'][:n]

0       00000001_000.png
1       00000001_001.png
2       00000001_002.png
3       00000002_000.png
4       00000003_001.png
              ...       
4994    00001335_002.png
4995    00001335_003.png
4996    00001335_004.png
4997    00001335_005.png
4998    00001335_006.png
Name: Image Index, Length: 4999, dtype: object

In [7]:
df1 = df[:n]

In [8]:
labels = ['Cardiomegaly',
 'Emphysema',
 'Effusion',
 'No Finding',
 'Hernia',
 'Infiltration',
 'Mass',
 'Nodule',
 'Atelectasis',
 'Pneumothorax',
 'Pleural_Thickening',
 'Pneumonia',
 'Fibrosis',
 'Edema',
 'Consolidation']

In [9]:
for l in labels:
    df1[l] = 0
    
for i in range(len(df1.index)):
    for j in range(len(labels)):
        label = labels[j]
        if label in df1.loc[i,'Finding Labels'].split('|'):
            df1.loc[i, label] = 1

/Users/chamusyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/chamusyuan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [10]:
df1.to_csv('preprocessed.csv')

In [11]:
def preprocess_NIH(split):
    split['Patient Age'] = np.where(split['Patient Age'].between(0,19), 19, split['Patient Age'])
    split['Patient Age'] = np.where(split['Patient Age'].between(20,39), 39, split['Patient Age'])
    split['Patient Age'] = np.where(split['Patient Age'].between(40,59), 59, split['Patient Age'])
    split['Patient Age'] = np.where(split['Patient Age'].between(60,79), 79, split['Patient Age'])
    split['Patient Age'] = np.where(split['Patient Age']>=80, 81, split['Patient Age'])
   
    copy_sunbjectid = split['Patient ID']
    split.drop(columns = ['Patient ID'])
   
    split = split.replace([[None], -1, "[False]", "[True]", "[ True]"],
                            [0, 0, 0, 1, 1])
    
    split['Patient Age'] = split['Patient Age'].replace([19, 39, 59, 79, 81],
                            [ "0-20", "20-40", "40-60", "60-80", "80-"])
   
    split['subject_id'] = copy_sunbjectid
    split['Sex'] = split['Patient Gender']
    split['Age'] = split['Patient Age']
    split['path'] = split['Image Index']
    split = split.drop(columns=["Patient Gender", 'Patient Age', 'Image Index'])
   
    return split

In [12]:
data = preprocess_NIH(df1)

/Users/chamusyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/chamusyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/chamusyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [13]:
np.random.seed(0)

In [14]:
def random_split(df, split_portion):
    
    total_patient_id = pd.unique(df['Patient ID'])
    total_patient_id = pd.DataFrame(data=total_patient_id, columns=['Patient ID'])
    total_patient_id['random_number'] = np.random.uniform(size=len(total_patient_id))

    train_id = total_patient_id[total_patient_id['random_number'] <= split_portion[0]]
    valid_id = total_patient_id[(total_patient_id['random_number'] > split_portion[0]) & (total_patient_id['random_number'] <= split_portion[1])]
    test_id = total_patient_id[total_patient_id['random_number'] > split_portion[1]]

    train_id = train_id.drop(columns=['random_number'])
    valid_id = valid_id.drop(columns=['random_number'])
    test_id = test_id.drop(columns=['random_number'])

    train_df = train_id.merge(df, left_on="Patient ID", right_on="Patient ID")
    valid_df = valid_id.merge(df, left_on="Patient ID", right_on="Patient ID")
    test_df = test_id.merge(df, left_on="Patient ID", right_on="Patient ID")

    print(len(train_df))
    print(len(valid_df))
    print(len(test_df))

    train_df.to_csv("train.csv", index=False)
    valid_df.to_csv("valid.csv", index=False)
    test_df.to_csv("test.csv", index=False)

In [15]:
random_split(data, [0.8,0.9])

3854
507
638
